# PyKX Example
Query tick data thru gateway from pyKX

**WORK IN PROGRESS, NOT COMPLETED**

In [1]:
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *
from env2 import *

clusters = {
    "rdb_conn": "RDB_basictickdb_20230601",
    "hdb_conn": "HDB_basictickdb_20230601",
    "gw_conn" : "GATEWAY_20230601"
}

# where is the license from?
os.getenv("QHOME")


'/home/ec2-user/anaconda3/lib/python3.9/site-packages/pykx/lib'

In [2]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


In [3]:
conns = {}

for k, v in clusters.items():
    try:
        conn_str = get_kx_connection_string(client, environmentId=ENV_ID, clusterName=v, userName=KDB_USERNAME, boto_session=session)
    except:
        conn_str = ""

    conns[k] = conn_str
    

In [4]:
conns

{'rdb_conn': ':tcps://vpce-011b9e5db9449efe9-ssju6ad9.vpce-svc-0c382efe0ad012fe4.us-east-1.vpce.amazonaws.com:443:bob:Host=vpce-011b9e5db9449efe9-ssju6ad9.vpce-svc-0c382efe0ad012fe4.us-east-1.vpce.amazonaws.com&Port=5000&User=bob&Action=finspace%3AConnectKxCluster&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEIz%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIH49%2Fdu%2FAOC2ZqRBlgzNAkriM8yxjmBiWESc0xIuB1R%2FAiEA0JHO7IXorpRLcbgbVupyRRKKc7n164tzR3XgGgAba3oqgAMIxP%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARAAGgw2MTI4NDEzODM1OTQiDHarf70ATvP9jNzT0CrUAqQGx24OMgNpqOisPiuwBllhkN8Cteik%2FReUMjXyXay2IVaRP4BSd3FJ%2FJAtVKMwLWvxf4iR%2BKZAMAHLHAaSHaCNEYAc7RMl0%2BJGRH6fPuDDhIkdYh%2BC2CtY089p8Mj9cuytZbVwW%2B6LkfVqIXxj0a9yMfLmBo016fLqjo3HBGK1wReb2Q23c9qhPppT1lGUeO8ccz6W938pWtEvAT6InND7MJH69UDk5TFocq7el05NlNdA76op8CBwqxLgyaCwKZqnsNBz4sPT4gEgUNEKWzz9u2iwz4nWOw0Xtobp03hRfMGj8j%2B5rWbmsWsuhFIzOmrSJoh1KMJQ%2FB32RrMZSVZghv3q%2FED%2Fr9MYy7yZzjTRcug8mQe8GGdwUOOe4Scqh6McJzyMZm%2BhJwGnJdhFBVpC6HuHQCzny%2FjgQ8OIaMPItuWM7OsSsIO1

# Connect and Query

In [5]:
hdb_name = clusters['hdb_conn']

# Query the HDB
hdb = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=hdb_name, 
                          userName=KDB_USERNAME, boto_session=session)

In [6]:
hdb("tables[]").py()

['example']

In [7]:
# Tables
tables = hdb("tables[]").py()
print(f"Tables ({len(tables)}): {tables}")

# Schema
schema_pdf = hdb("meta `example").pd()
display(schema_pdf)


Tables (1): ['example']


,t,f,a
c,,,
date,b'd',,
sym,b's',,p
time,b'p',,
number,b'j',,


In [8]:
# Simple Query
res_table = hdb("select counts:count i, avg_num: avg number by date from example").pd()
display(res_table)

# Number of Rows
rows = hdb("count example").py()
print(f"Rows: {rows:,}")

,counts,avg_num
date,,
2023-04-14,1000000,499800.464405
2023-04-15,1000000,499865.008159
2023-04-16,1000000,499912.379127
2023-04-17,1000000,500078.393386
2023-04-18,1000000,500264.684412
2023-04-19,1000000,499849.962912
2023-04-20,1000000,500082.257829
2023-04-21,1000000,500169.492354
2023-04-22,1000000,499903.047577


Rows: 13,896,856
